# OpenAPI agents

We can construct agents to consume arbitrary APIs, here APIs conformant to the OpenAPI/Swagger specification.

# 1st example: hierarchical planning agent

In this example, we'll consider an approach called hierarchical planning, common in robotics and appearing in recent works for LLMs X robotics. We'll see it's a viable approach to start working with a massive API spec AND to assist with user queries that require multiple steps against the API.

The idea is simple: to get coherent agent behavior over long sequences behavior & to save on tokens, we'll separate concerns: a "planner" will be responsible for what endpoints to call and a "controller" will be responsible for how to call them.

In the initial implementation, the planner is an LLM chain that has the name and a short description for each endpoint in context. The controller is an LLM agent that is instantiated with documentation for only the endpoints for a particular plan. There's a lot left to get this working very robustly :)

---

## To start, let's collect some OpenAPI specs.

In [29]:
import os, yaml

In [30]:
!wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
!mv openapi.yaml openai_openapi.yaml
!wget https://www.klarna.com/us/shopping/public/openai/v0/api-docs
!mv api-docs klarna_openapi.yaml
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

--2023-04-08 19:18:55--  https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122995 (120K) [text/plain]
Saving to: ‘openapi.yaml’

openapi.yaml        100%[===================>] 120.11K  --.-KB/s    in 0.01s   

2023-04-08 19:18:55 (9.03 MB/s) - ‘openapi.yaml’ saved [122995/122995]

--2023-04-08 19:18:56--  https://www.klarna.com/us/shopping/public/openai/v0/api-docs
Resolving www.klarna.com (www.klarna.com)... 52.84.150.46, 52.84.150.34, 52.84.150.37, ...
Connecting to www.klarna.com (www.klarna.com)|52.84.150.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘api-docs’

api-docs                [ <=>                ]  

In [31]:
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

In [32]:
with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)
    
with open("klarna_openapi.yaml") as f:
    raw_klarna_api_spec = yaml.load(f, Loader=yaml.Loader)
klarna_api_spec = reduce_openapi_spec(raw_klarna_api_spec)

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

---

We'll work with the Spotify API as one of the examples of a somewhat complex API. There's a bit of auth-related setup to do if you want to replicate this.

- You'll have to set up an application in the Spotify developer console, documented [here](https://developer.spotify.com/documentation/general/guides/authorization/), to get credentials: `CLIENT_ID`, `CLIENT_SECRET`, and `REDIRECT_URI`.
- To get an access tokens (and keep them fresh), you can implement the oauth flows, or you can use `spotipy`. If you've set your Spotify creedentials as environment variables `SPOTIPY_CLIENT_ID`, `SPOTIPY_CLIENT_SECRET`, and `SPOTIPY_REDIRECT_URI`, you can use the helper functions below:

In [33]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env', override=True)

os.environ['SPOTIPY_CLIENT_ID']

'f02db32129af4e01a38edbcc514ea591'

In [34]:
import spotipy.util as util
from langchain.requests import RequestsWrapper

def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(raw_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']['scopes'].keys())
    access_token = util.prompt_for_user_token(scope=','.join(scopes))
    return {
        'Authorization': f'Bearer {access_token}'
    }

# Get API credentials.
headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

## How big is this spec?

In [35]:
endpoints = [
    (route, operation)
    for route, operations in raw_spotify_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

63

In [36]:
import tiktoken
enc = tiktoken.encoding_for_model('text-davinci-003')
def count_tokens(s): return len(enc.encode(s))

count_tokens(yaml.dump(raw_spotify_api_spec))

80326

In [42]:
from tqdm import tqdm
eval_dataframe = ['hi1', 'bye1']
eval_dataframe2 = ['hi', 'bye']
for question, ans in tqdm(zip(eval_dataframe, eval_dataframe2),
                              desc='Running OpenAssistant',
                              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'):
  print(question)
  print(ans)

Running OpenAssistant: |          | 2/? [00:00<00:00, 26630.50it/s]

hi1
hi
bye1
bye


# 3.5 turbo



In [39]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
# llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
llm = OpenAI(model_name="gpt-4", temperature=0.0)

/Users/kastanday/miniforge3/envs/openai/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/kastanday/miniforge3/envs/openai/lib/python3.10/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [40]:
spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)
user_query = "make me a playlist with the song I would most like, based on my recommendations from spotify."
spotify_agent.run(user_query)
# ValueError: Could not parse LLM output: `Before starting the plan, I need to know the user_id and playlist_id to use in the POST requests. I also need to decide whether to use the "artists" or "tracks" type for the top API call.

def _get_next_action(self, full_inputs: Dict[str, str]) -> AgentAction:
  full_output = self.llm_chain.predict(**full_inputs)
  parsed_output = self._extract_tool_and_input(full_output)




> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the user's top recommended song from Spotify and create a playlist with that song.
Observation: 1. GET /me to get the current user's information
2. GET /recommendations to get the top recommended songs for the user
3. GET /users/{user_id} to get the user's public profile information
4. POST /users/{user_id}/playlists to create a new playlist for the user
5. POST /playlists/{playlist_id}/tracks to add the top recommended song to the newly created playlist
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /me to get the current user's information
2. GET /recommendations to get the top recommended songs for the user
3. GET /users/{user_id} to get the user's public profile information
4. POST /users/{user_id}/playlists to create a new playlist for the user
5. POST /playlists/{playlist_id}/tracks to add the top recommended song to the newl

ValueError: Could not parse LLM output: `I am finished executing a plan and have the information the user asked for or the data the user asked to create.`

## Let's see some examples!

Starting with GPT-4. (Some robustness iterations under way for GPT-3 family.)

In [26]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
llm = OpenAI(model_name="gpt-4", temperature=0.0)

/Users/kastanday/miniforge3/envs/openai/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/kastanday/miniforge3/envs/openai/lib/python3.10/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [28]:
spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)
user_query = "make me a playlist with the oldest album made by bruce springsteen."
spotify_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to create a playlist with the oldest album made by Bruce Springsteen
Observation: 1. GET /search to search for the artist "Bruce Springsteen"
2. GET /artists/{id}/albums to get a list of albums by Bruce Springsteen
3. Sort the albums by release date and select the oldest album
4. GET /albums/{id}/tracks to get the tracks from the oldest album
5. GET /me to get the current user's ID
6. POST /users/{user_id}/playlists to create a new playlist for the user
7. POST /playlists/{playlist_id}/tracks to add the tracks from the oldest album to the newly created playlist
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /search to search for the artist "Bruce Springsteen"
2. GET /artists/{id}/albums to get a list of albums by Bruce Springsteen
3. Sort the albums by release date and select the oldest album
4. GET /albums/{id}/tracks to get the tracks f

'I have created a playlist with the oldest album made by Bruce Springsteen. The playlist ID is 4PonCgI52Y2LQGTwlQzFo5.'

In [ ]:
ValueError: Could not parse LLM output: `I cannot find the earliest album of Bruce Springsteen with the information I have. I need more information about his albums to proceed with the plan.`

In [13]:
user_query = "give me a song I'd like, something like Sure Thing by Miguel"
spotify_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get a song recommendation based on Sure Thing by Miguel
Observation: 1. GET /search to search for the track "Sure Thing" by Miguel and obtain its Spotify ID.
2. GET /recommendations with the obtained track ID as a seed to generate a list of recommended songs based on "Sure Thing" by Miguel.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /search to search for the track "Sure Thing" by Miguel and obtain its Spotify ID.
2. GET /recommendations with the obtained track ID as a seed to generate a list of recommended songs based on "Sure Thing" by Miguel.

> Entering new AgentExecutor chain...
Action: requests_get
Action Input: {"url": "https://api.spotify.com/v1/search?q=track:Sure%20Thing%20artist:Miguel&type=track", "output_instructions": "Extract the Spotify ID of the first track in the search results"}
Observation: 0JXXNGljqupsJaZsgSbMZV

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e86bcfc38711e520f3f5cd3f3ac4c9b2 in your message.).


I am finished executing the plan and have the information the user asked for.
Final Answer: Here are two recommended songs based on "Sure Thing" by Miguel:
1. Daniel Caesar - 20wkVLutqVOYrc0kxFs7rA
2. Bryson Tiller - 2EMAnMvWE2eb56ToJVfCWs

> Finished chain.


'Here are two recommended songs based on "Sure Thing" by Miguel:\n1. Daniel Caesar - 20wkVLutqVOYrc0kxFs7rA\n2. Bryson Tiller - 2EMAnMvWE2eb56ToJVfCWs'

#### Try another API.


In [16]:
headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
}
openai_requests_wrapper=RequestsWrapper(headers=headers)

In [ ]:
# Meta!  # "generate a short piece of advice"

# llm = OpenAI(model_name="gpt-4", temperature=0.25)
# openai_agent = planner.create_openapi_agent(openai_api_spec, openai_requests_wrapper, llm)
# user_query = "generate a short piece of advice"
# openai_agent.run(user_query)

Takes awhile to get there!

## 2nd example: "json explorer" agent

Here's an agent that's not particularly practical, but neat! The agent has access to 2 toolkits. One comprises tools to interact with json: one tool to list the keys of a json object and another tool to get the value for a given key. The other toolkit comprises `requests` wrappers to send GET and POST requests. This agent consumes a lot calls to the language model, but does a surprisingly decent job.


In [14]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

In [17]:
with open("openai_openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec=JsonSpec(dict_=data, max_value_length=4000)


openapi_toolkit = OpenAPIToolkit.from_llm(OpenAI(temperature=0), json_spec, openai_requests_wrapper, verbose=True)
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    verbose=True
)

In [18]:
openapi_agent_executor.run("Make a post request to openai /completions. The prompt should be 'tell me a joke.'")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, get the value directly.')
Thought: I should get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the base url for the API
Final Answer: The base url for the API is https://api.openai.com/v1

> Finished chain.

Observation: The base url for the API is https://api.openai.com/v1
Thought: I should find the path for the /completions endpoint.
Action: json_explorer
Action Input: What is the path for th

'"By the way, do you tell me a joke!"'